In [7]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")


In [8]:
Valuation_Multiple = 33
df = pd.read_csv('FRPcreditCampaignAllocation.csv')

In [9]:
df.head(5)

,Campaign ID,Campaign Create Date,Campaign Start Date,Credit ID,Credit Type,Company ID,Broker UID,Brokerage,Credit Consumption Date,Discount Code,Revenue
0,ca7c9d2b-0dd4-497e-958b-7505d5edd2bf,8/5/24,9/1/24,27784756-efde-4479-aff7-d4b8f92cf9e1,Digital Profile,71eee226-9e8c-4b4c-b59b-806a75e8cb5e,90a1570e-1822-4815-adc2-f00f53432e77,FRP,8/5/24,1,0
1,ca7c9d2b-0dd4-497e-958b-7505d5edd2bf,8/5/24,9/1/24,28c9113f-d716-4cd3-b6c7-3a861b88b16c,Digital Profile,c0a2771c-5bb1-4d9a-b085-0a0819a3268c,cc197feb-8f33-4a4a-aa3f-e21bdfda7276,FRP,8/5/24,1,0
2,ca7c9d2b-0dd4-497e-958b-7505d5edd2bf,8/5/24,9/1/24,3d9d62bd-23c1-46bb-b3f7-bffa26bd9e0d,Digital Profile,e5fec5cd-9a44-405f-9a32-55b55c21c8b7,de02b83f-15d5-4a82-9693-ea77b5622106,FRP,8/5/24,0,209
3,ca7c9d2b-0dd4-497e-958b-7505d5edd2bf,8/5/24,9/1/24,8aa817d9-593f-4462-ac17-9a138818286b,Digital Profile,5efb09a4-431c-4948-9b74-9c86ff3bdc78,9e382f0b-112f-4991-8250-fbf3da954b62,FRP,8/5/24,1,0
4,ca7c9d2b-0dd4-497e-958b-7505d5edd2bf,8/5/24,9/1/24,5b261598-a765-422f-a7ab-f3d331b9c68d,Digital Profile,bc17d946-7030-491d-ac88-fe37562a9da0,eb35031e-606b-4ca2-aaeb-69017bc549dc,FRP,8/5/24,1,0


In [10]:
# List the column names
column_names = df.columns
print(column_names)

Index(['Campaign ID', 'Campaign Create Date', 'Campaign Start Date',
       'Credit ID', 'Credit Type', 'Company ID', 'Broker UID', 'Brokerage',
       'Credit Consumption Date', 'Discount Code', 'Revenue'],
      dtype='object')


In [11]:
#Brokerage is the distinct brokerage values from df

#MRR Reporting Period is the distinct MRR Reporting Period values from df

#Digital Profile Credit Revenue Per User calculates the sum of ‘Revenue’  where ‘Credit Type’ == ‘Digital Profile’ divided by the distinct count of ‘Broker UID’ grouped by ‘MRR Reporting Period’

#Placement Credit Revenue Per User calculates the sum of ‘Revenue’  where ‘Credit Type’ == ‘Placement’ divided by the distinct count of ‘Broker UID’ grouped by ‘MRR Reporting Period’

#Total MRR Per User calculates the sum of ‘Revenue’  divided by the distinct count of ‘Broker UID’ grouped by ‘MRR Reporting Period’

#Monthly Active User Count is the count of distinct ‘Broker UID’ grouped by MRR Reporting Period

#MRR is ‘Total MRR Per User’ * ‘Monthly Active User Count’

In [12]:
# Convert 'Credit Consumption Date' to datetime and extract 'MRR Reporting Period'
df['Credit Consumption Date'] = pd.to_datetime(df['Credit Consumption Date'])
df['MRR Reporting Period'] = df['Credit Consumption Date'].dt.to_period('M')

# Creating the new dataframe MRR with the required data elements
grouped = df.groupby(['Brokerage', 'MRR Reporting Period'])
digital_profile_revenue_per_user = grouped.apply(lambda x: x[x['Credit Type'] == 'Digital Profile']['Revenue'].sum() / x['Broker UID'].nunique())
placement_credit_revenue_per_user = grouped.apply(lambda x: x[x['Credit Type'] == 'Placement Credit']['Revenue'].sum() / x['Broker UID'].nunique())
total_mrr_per_user = grouped['Revenue'].sum() / grouped['Broker UID'].nunique()
monthly_active_user_count = grouped['Broker UID'].nunique()

mrr_data = {
    'Brokerage': digital_profile_revenue_per_user.index.get_level_values('Brokerage'),
    'MRR Reporting Period': digital_profile_revenue_per_user.index.get_level_values('MRR Reporting Period'),
    'Digital Profile Credit Revenue Per User': digital_profile_revenue_per_user.values,
    'Placement Credit Revenue Per User': placement_credit_revenue_per_user.values,
    'Total MRR Per User': total_mrr_per_user.values,
    'Monthly Active User Count': monthly_active_user_count.values,
    'Consumption MRR': total_mrr_per_user.values * monthly_active_user_count.values
}


mrr_df = pd.DataFrame(mrr_data)
mrr_df['Subscription MRR'] = 65000/12 + 200000/12
mrr_df['MRR'] = mrr_df['Subscription MRR']+mrr_df['Consumption MRR']
mrr_df['ARR Run Rate'] = mrr_df['MRR']*12
mrr_df['Notional Valuation'] = mrr_df['ARR Run Rate']*Valuation_Multiple
mrr_df

,Brokerage,MRR Reporting Period,Digital Profile Credit Revenue Per User,Placement Credit Revenue Per User,Total MRR Per User,Monthly Active User Count,Consumption MRR,Subscription MRR,MRR,ARR Run Rate,Notional Valuation
0,FRP,2024-08,257.443709,0.000000,257.443709,151,38874.0,22083.333333,60957.333333,731488.0,24139104.0
1,FRP,2024-09,583.758621,0.000000,583.758621,203,118503.0,22083.333333,140586.333333,1687036.0,55672188.0
2,FRP,2024-10,1231.218009,21.729858,1252.947867,211,264372.0,22083.333333,286455.333333,3437464.0,113436312.0
3,FRP,2024-11,435.061224,88.224490,523.285714,196,102564.0,22083.333333,124647.333333,1495768.0,49360344.0
4,FRP,2024-12,0.000000,262.818750,262.818750,160,42051.0,22083.333333,64134.333333,769612.0,25397196.0
5,FRP,2025-01,0.000000,231.894410,231.894410,161,37335.0,22083.333333,59418.333333,713020.0,23529660.0
6,FRP,2025-02,0.000000,163.213115,163.213115,61,9956.0,22083.333333,32039.333333,384472.0,12687576.0
7,FRP,2025-03,0.000000,131.000000,131.000000,2,262.0,22083.333333,22345.333333,268144.0,8848752.0


In [13]:
mrr_df.to_csv('mrr.csv', index=False)